# SINGA Model Classes

<img src="http://singa.apache.org/en/_static/images/singav1-sw.png" width="500px"/>

## Layer

Typically, the life cycle of a layer instance includes:
 1. construct layer without input_sample_shapes, goto 2;
    construct layer with input_sample_shapes, goto 3;
 2. call setup to create the parameters and setup other meta fields
 3. call forward or access layer members
 4. call backward and get parameters for update

In [5]:
from singa import tensor, device, layer

# help(Layer)
layer.engine='singacpp'

## Common layers

In [ ]:
from singa.layer import Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, Softmax

### Dense Layer

In [ ]:
dense = Dense('dense', 3, input_sample_shape=(2,))
#dense.param_names()
w, b = dense.param_values()
print w.shape, b.shape

In [ ]:
w.gaussian(0, 0.1)
b.set_value(0)

In [ ]:
x  = tensor.Tensor((2,2))
x.uniform(-1, 1)
y = dense.forward(True, x)
tensor.to_numpy(y)


In [ ]:
gx, [gw, gb] = dense.backward(True, y)
print gx.shape, gw.shape, gb.shape

### Convolution Layer

In [ ]:
conv = Conv2D('conv', 4, 3, 1, input_sample_shape=(3, 6, 6))
print conv.get_output_sample_shape()

### Pooling Layer

In [ ]:
pool = MaxPooling2D('pool', 3, 2, input_sample_shape=(4, 6, 6))
print pool.get_output_sample_shape()

## Branch layers

In [1]:
from singa.layer import Split, Merge, Slice, Concat

In [ ]:
split = Split('split', 2, input_sample_shape=(4, 6, 6))
print split.get_output_sample_shape()

In [ ]:
merge = Merge('merge', input_sample_shape=(4, 6, 6))
print merge.get_output_sample_shape()

In [2]:
sli = Slice('slice', 1, [2], input_sample_shape=(4, 6, 6))
# print sli.get_output_sample_shape()

TypeError: GetOutputSampleShape() takes exactly 1 argument (2 given)

In [3]:
concat = Concat('concat', 1, input_sample_shapes=[(3, 6, 6), (1, 6, 6)])
print concat.get_output_sample_shape()

(4L, 6L, 6L)


## Metric and Loss

In [13]:
from singa import metric
import numpy as np

x = tensor.Tensor((3, 5))
x.uniform(0, 1)  # randomly genearte the prediction activation
x = tensor.softmax(x)  # normalize the prediction into probabilities
print tensor.to_numpy(x)
y = tensor.from_numpy(np.array([0, 1, 3], dtype=np.int))  # set the truth

f = metric.Accuracy()
acc = f.evaluate(x, y)  # averaged accuracy over all 3 samples in x
print acc

[[ 0.26027831  0.1504543   0.24347362  0.14403342  0.20176032]
 [ 0.17135921  0.22572449  0.27182052  0.18477756  0.14631821]
 [ 0.27586439  0.206112    0.25757927  0.14359026  0.11685407]]
0.333333343267


In [16]:
from singa import loss

x = tensor.Tensor((3, 5))
x.uniform(0, 1)  # randomly genearte the prediction activation
y = tensor.from_numpy(np.array([0, 1, 3], dtype=np.int))  # set the truth

f = loss.SoftmaxCrossEntropy()
l = f.forward(True, x, y)  # l is tensor with 3 loss values
g = f.backward()  # g is a tensor containing all gradients of x w.r.t l
print l.l1()
print tensor.to_numpy(g)

1.65742790699
[[-0.83755541  0.28364336  0.13934435  0.26742008  0.14714769]
 [ 0.23799847 -0.83042568  0.19956587  0.14701514  0.24584624]
 [ 0.16164713  0.12167476  0.26101682 -0.7485224   0.20418374]]


## Optimizer

In [17]:
from singa import optimizer

sgd = optimizer.SGD(lr=0.01, momentum=0.9, weight_decay=1e-4)
p = tensor.Tensor((3,5))
p.uniform(-1, 1)
g = tensor.Tensor((3,5))
g.gaussian(0, 0.01)

sgd.apply(1, g, p, 'param')  # use the global lr=0.1 for epoch 1
sgd.apply_with_lr(2, 0.03, g, p, 'param')  # use lr=0.03 for epoch 2

## FeedForwardNet

In [21]:
from singa import net as ffnet
layer.engine = 'singacpp'
net = ffnet.FeedForwardNet(loss.SoftmaxCrossEntropy(), metric.Accuracy())
net.add(layer.Conv2D('conv1', 32, 5, 1, input_sample_shape=(3,32,32,)))
net.add(layer.Activation('relu1'))
net.add(layer.MaxPooling2D('pool1', 3, 2))
net.add(layer.Flatten('flat'))
net.add(layer.Dense('dense', 10))

# init parameters
for p in net.param_values():
    if len(p.shape) == 0:
        p.set_value(0)
    else:
        p.gaussian(0, 0.01)
print net.param_names()

conv1 (32L, 32L, 32L)
relu1 (32L, 32L, 32L)
pool1 (32L, 16L, 16L)
flat (8192L,)
dense (10L,)
[u'conv1_weight', u'conv1_bias', u'dense_weight', u'dense_bias']


In [ ]:
layer.engine = 'cudnn'
net = ffnet.FeedForwardNet(loss.SoftmaxCrossEntropy(), metric.Accuracy())
net.add(layer.Conv2D('conv1', 32, 5, 1, input_sample_shape=(3,32,32,)))
net.add(layer.Activation('relu1'))
net.add(layer.MaxPooling2D('pool1', 3, 2))
net.add(layer.Flatten('flat'))
net.add(layer.Dense('dense', 10))

# init parameters
for p in net.param_values():
    if len(p.shape) == 0:
        p.set_value(0)
    else:
        p.gaussian(0, 0.01)
        
# move net onto gpu
dev = device.create_cuda_gpu()
net.to_device(dev)